In [4]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import psycopg2
from psycopg2 import sql
import json 

In [6]:
""" FUNCIONES """



# 1. Conexión con BD
def conexionBDPostgresSQL():
    # Se omiten los datos de conexión.
    conexion = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="serRIB99",
        host="localhost",
        port="5432"
    )
    return conexion

# 1.2 Insertar datos Generación
def Demanda_Preprocesado_InsertBD_SQLALCHEMY(dataframeDemanda):
  # Datos de conexión a POSTGRESQL
  usuario = 'postgres'
  password = 'serRIB99'
  servidor = 'localhost'  
  puerto = '5432'
  basedatos = 'postgres'

  # Crear la URL de conexión
  urlConexion = f'postgresql+psycopg2://{usuario}:{password}@{servidor}:{puerto}/{basedatos}'
  # Motor conexión Postgresql
  engine = create_engine(urlConexion)

  # Tabla y Esquema de BD
  esquemaBD = 'tfm'
  tabla = 'demanda_procesada'

  # Insertar en BBDD desde un Dataframe
  dataframeDemanda.to_sql(tabla, engine, schema=esquemaBD, if_exists='append', index=False)

# 3. Función para ejecutar cualquier comando SQL
def ejecutarComandoSQL(comandoSQL):
  connection = conexionBDPostgresSQL()
  if connection is None:
      return
  try:
      cursor = connection.cursor()
      cursor.execute(comandoSQL)
      connection.commit()
      print(f"Comando ejecutado con éxito: {comandoSQL[0:50]}")
  except Exception as error:
      print(f"Error al ejecutar el comando: {error}")
  finally:
      if connection:
          cursor.close()
          connection.close()

# 2. # Obtener datos generación desde BBDD y generar Dataframe y CSV
def Preprocesamiento_DemandaEnergia_ToDataframe():
  # Consultar diferentes tipos de energía
  comandoSQL = """
    SELECT fecha, valordemanda
    FROM tfm.demanda d
    WHERE d.fecha BETWEEN '2011-01-01' AND '2024-12-31'
  """
  registrosDemanda = ejecutarComandoSQLSelect(comandoSQL)

  # Diccionario dónde insertar los datos
  datosDemanda = {
      'fecha': [],
      'valordemanda': []
  }

  # Procesar los resultados y agregar los datos al diccionario
  for registro in registrosDemanda:
    datosDemanda['fecha'].append(registro[0])
    datosDemanda['valordemanda'].append(registro[1])
    
  # Crear dataframe desde "Datos Generación"
  dataframeDemanda = pd.DataFrame(datosDemanda)

  # DataFrame a CSV
  dataframeDemanda.to_csv('Demandaprocesada.csv', index=False)

  # Insertar en BD
  Demanda_Preprocesado_InsertBD_SQLALCHEMY(dataframeDemanda)

  
# 4. Función para ejecutar cualquier comando SQL (SELECT)
def ejecutarComandoSQLSelect(comandoSQL):
  connection = conexionBDPostgresSQL()
  resultados = []
  if connection is None:
      return
  try:
      cursor = connection.cursor()
      cursor.execute(comandoSQL)
      # Obtener todos los resultados de la consulta
      resultados = cursor.fetchall()

      # Imprimir los resultados
      """for fila in resultados:
          print(fila)"""
      connection.commit()
      print(f"Comando ejecutado con éxito: {comandoSQL[0:50]}")
  except Exception as error:
      print(f"Error al ejecutar el comando: \n{error}")
  finally:
    if connection:
        cursor.close()
        connection.close()
    return resultados




In [9]:
""" MAIN """

ejecutarComandoSQL("TRUNCATE TABLE tfm.demanda_procesada;")

Preprocesamiento_DemandaEnergia_ToDataframe()

Comando ejecutado con éxito: TRUNCATE TABLE tfm.demanda_procesada;
Comando ejecutado con éxito: 
    SELECT fecha, valordemanda
    FROM tfm.deman
